Ref: https://arxiv.org/pdf/1711.10561

# Equação de Burgers

$$\left\{\begin{array}{l}
u_t + u u_x - (0.01/\pi) u_{xx} = 0, \ x \in [-1,1], \ t \in [0,1] \\
u(x,0) = -\sin(\pi x) \\
u(-1,t) = u(1,t) = 0
\end{array}\right.$$

#### Importando Bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch as tc

from torch import nn
from torch.optim.lr_scheduler import StepLR

#### Definindo Dados de treino

In [ ]:
tc.manual_seed(21)

# Definindo intervalo 
t_i = 0
t_f = 1
x_i = -1
x_f = 1

### ================================== ###
###    Condições Iniciais/Contorno     ###
### ================================== ###

# Número de pontos
n_pontos = 200

# Pontos analíticos aleatórios
t = tc.linspace(t_i, t_f, n_pontos*10)
x = tc.linspace(x_i, x_f, n_pontos*10)

rand_indices = tc.randint(0, n_pontos*10, (n_pontos,))
t_treino_CI = t[rand_indices].reshape(-1, 1)
x_treino_CI = x[rand_indices].reshape(-1, 1)
t_treino_CI.requires_grad_(True)
x_treino_CI.requires_grad_(True)

### ================================== ###
###           Dados para EDP           ###
### ================================== ###

# Hipercubo Latino

# Número de pontos
n_pontos = 1000





In [ ]:
# ================================== #
#     Geração do Hipercubo Latino    #
# ================================== #

# Domínios do problema
x_min, x_max = -1.0, 1.0
t_min, t_max = 0.0, 1.0

# Número de pontos (definido anteriormente)
# n_pontos = 1000

# Gerando hipercubo latino para 2D (x, t)
tc.manual_seed(42)  # Para reprodutibilidade

# Pontos iniciais aleatórios no hipercubo
x_latin = tc.rand(n_pontos) * (x_max - x_min) + x_min
t_latin = tc.rand(n_pontos) * (t_max - t_min) + t_min

# Hipercubo latino: ordena x e embaralha t para maximizar cobertura
idx_sort = tc.argsort(x_latin)
x_latin_sorted = x_latin[idx_sort]
t_latin_sorted = t_latin[idx_sort]

# Embaralha t para boa dispersão no tempo
perm = tc.randperm(n_pontos)
t_latin = t_latin_sorted[perm]
x_latin = x_latin_sorted[perm]

# Convertendo para tensores com gradiente para a EDP (interior)
x_interior = x_latin.clone().detach().requires_grad_(True).reshape(-1, 1)
t_interior = t_latin.clone().detach().requires_grad_(True).reshape(-1, 1)

# u_interior será calculado pela rede, então não precisamos de valores target aqui
print(f"Hipercubo latino gerado: {n_pontos} pontos")
print(f"  → x ∈ [{x_min:.1f}, {x_max:.1f}], t ∈ [{t_min:.1f}, {t_max:.1f}]")
print(f"  → Pontos interiores para EDP: {len(x_interior)} pontos")

# Preparando dados de contorno e inicial para supervisão
print("\nPreparando condições de contorno e inicial...")


In [ ]:
# ================================== #
#        Geração das Condições        #
# ================================== #

# Condições de contorno: u(-1,t) = u(1,t) = 0 para t ∈ [0,1]
n_contorno = 50
t_contorno = tc.linspace(t_min, t_max, n_contorno).reshape(-1, 1)
x_left = tc.full_like(t_contorno, x_min)
x_right = tc.full_like(t_contorno, x_max)
u_contorno = tc.zeros_like(t_contorno)  # u = 0 nas bordas

# Condições iniciais: u(x,0) = -sin(πx) para x ∈ [-1,1]
n_inicial = 50
x_inicial = tc.linspace(x_min, x_max, n_inicial).reshape(-1, 1)
t_inicial = tc.zeros_like(x_inicial)
u_inicial = -tc.sin(tc.pi * x_inicial)

print(f"Condições de contorno geradas: {n_contorno} pontos em cada borda (esquerda + direita)")
print(f"Condições iniciais geradas: {n_inicial} pontos em t=0")
print(f"Total de pontos supervisionados: {2*n_contorno + n_inicial}")

# Separando pontos interiores (para EDP) dos pontos de contorno/inicial
# Os pontos do hipercubo latino são todos interiores por definição

print("\nPontos separados:")
print(f"  → Interiores (EDP): {len(x_interior)} pontos")
print(f"  → Contorno (supervisão): {2*n_contorno} pontos") 
print(f"  → Inicial (supervisão): {n_inicial} pontos")


In [ ]:
# ================================== #
#          Plot dos Dados             #
# ================================== #

# Criando a malha para visualização de fundo
x_mesh = np.linspace(x_min, x_max, 100)
t_mesh = np.linspace(t_min, t_max, 100)
X, T = np.meshgrid(x_mesh, t_mesh)

# Solução analítica de Burgers para visualização de fundo (opcional)
# Como Burgers não tem solução fechada simples, usamos só a grade vazia
Z = np.zeros_like(X)  # Fundo vazio para destacar os pontos

plt.figure(figsize=(12, 8))

# Plot do fundo (grade vazia)
plt.contourf(X, T, Z, levels=20, cmap='RdBu_r', alpha=0.3)
plt.colorbar(label='u(x,t)')

# Plot dos pontos do hipercubo latino (azul) - EDP interior
plt.scatter(x_interior.detach().numpy(), t_interior.detach().numpy(), 
           c='blue', s=10, alpha=0.7, label=f'Pontos EDP (Hipercubo Latino, N={n_pontos})', 
           edgecolors='darkblue', linewidth=0.5)

# Plot das condições de contorno (vermelho) - esquerda x=-1
plt.scatter(x_left.detach().numpy(), t_contorno.detach().numpy(), 
           c='red', s=30, alpha=0.8, label=f'Contorno x=-1 (u=0, N={n_contorno})',
           marker='s', edgecolors='darkred', linewidth=1)

# Plot das condições de contorno (vermelho) - direita x=1
plt.scatter(x_right.detach().numpy(), t_contorno.detach().numpy(), 
           c='red', s=30, alpha=0.8, label=f'Contorno x=1 (u=0, N={n_contorno})',
           marker='s', edgecolors='darkred', linewidth=1)

# Plot das condições iniciais (roxo) - t=0
plt.scatter(x_inicial.detach().numpy(), t_inicial.detach().numpy(), 
           c='purple', s=40, alpha=0.9, label=f'Inicial t=0 (u=-sin(πx), N={n_inicial})',
           marker='^', edgecolors='darkviolet', linewidth=1.2)

# Configurações do plot
plt.xlabel('x', fontsize=12)
plt.ylabel('t', fontsize=12)
plt.title('Distribuição dos Pontos de Treino - Equação de Burgers\n(Azul: EDP Interior | Vermelho: Contorno | Roxo: Inicial)', 
          fontsize=14, fontweight='bold')
plt.xlim(x_min, x_max)
plt.ylim(t_min, t_max)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper right', fontsize=10, framealpha=0.9)
plt.tight_layout()

# Adicionando texto explicativo
plt.text(0.02, 0.98, 'Azul: Pontos interiores para EDP (hipercubo latino)\nVermelho: Condições de contorno Dirichlet (u=0)\nRoxo: Condição inicial u(x,0) = -sin(πx)', 
         transform=plt.gca().transAxes, fontsize=9, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.show()

print("Visualização dos pontos de treino:")
print(f"• Pontos azuis: {n_pontos} pontos do hipercubo latino para cálculo da EDP")
print(f"• Quadrados vermelhos: {2*n_contorno} pontos de contorno (x=-1 e x=1, u=0)")
print(f"• Triângulos roxos: {n_inicial} pontos iniciais (t=0, u=-sin(πx))")
print(f"• Total: {n_pontos + 2*n_contorno + n_inicial} pontos para o treinamento da PINN")
